In [40]:
import azureml.core
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.48


# Import Workspace and Datastores

In [41]:
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

#ws = Workspace.from_config("../config2.json")
print("Workspace {} is loaded".format(ws.name))

Workspace Auria is loaded


In [42]:
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type, ds.container_name, ds.account_name)

workspaceblobstore AzureBlob azureml-blobstore-963d160e-9bdd-40a8-852c-1fc06dfe7c7a auria5226806917
workspacefilestore AzureFile azureml-filestore-963d160e-9bdd-40a8-852c-1fc06dfe7c7a auria5226806917
videos AzureBlob videos happypathspublic
models AzureBlob styletransfer pipelinedata
breeds AzureBlob azureml-blobstore-315a742f-5c1f-465a-9486-7fd50bbb0012 danielscstoragebhmgfqha
images_datastore AzureBlob sampledata pipelinedata
auria5226806917__azureml_blobstore_963d160e_9bdd_40a8_852c_1fc06dfe7c7a AzureBlob azureml-blobstore-963d160e-9bdd-40a8-852c-1fc06dfe7c7a auria5226806917
auria9030644229__azureml_blobstore_af014520_168f_4d68_ab72_dbc070080caf AzureBlob azureml-blobstore-af014520-168f-4d68-ab72-dbc070080caf auria9030644229
auria5226806917__revisions AzureBlob revisions auria5226806917
minxia2storagexujmxsjn__azureml_blobstore_4e82210c_3a29_4f15_b59e_aabe71978be3 AzureBlob azureml-blobstore-4e82210c-3a29-4f15-b59e-aabe71978be3 minxia2storagexujmxsjn


# Datastore account - auria5226806917

# Datastore name (Blob) - azureml-blobstore-963d160e-9bdd-40a8-852c-1fc06dfe7c7a

# Datastore name (File) - azureml-filestore-963d160e-9bdd-40a8-852c-1fc06dfe7c7a

# List out the compute engines available 

In [43]:
cts = ws.compute_targets
for ct in cts:
    print(ct)

testing6854
demo-dogbreeds
adftest
gpuclusterNCv2
gpuclusterNCv3
cpucluster
v100cluster1
gpucluster
cpu-cluster
gpu-cluster


# Setting compute target with each compute engine

# Using gpuclusterNCv2 for compute engine

In [44]:
from azureml.core.compute import AmlCompute

# CPU Cluster
cpu_compute_target = "cpucluster"

try:
    cpu_compute = AmlCompute(ws, cpu_compute_target)
    print("Found existing compute target: {}".format(cpu_compute.name))
except:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 0,
                                                                max_nodes = 4)
    cpu_compute = AmlCompute.create(ws, cpu_compute_target, provisioning_config)
    cpu_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    print("Created new compute target: {}".format(cpu_compute.name))

# GPU Cluster
gpu_compute_target_1 = "gpuclusterNCv2"

try:
    gpu_compute_1 = AmlCompute(ws, gpu_compute_target_1)
    print("Found existing compute target: {}".format(gpu_compute_1.name))
except:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6S_V2",
                                                                min_nodes = 0,
                                                                max_nodes = 4)
    gpu_compute_1 = AmlCompute.create(ws, gpu_compute_target_1, provisioning_config)
    gpu_compute_1.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    print("Created new compute target: {}".format(gpu_compute_1.name))
    
    
gpu_compute_target_2 = "gpuclusterNCv3"

try:
    gpu_compute_2 = AmlCompute(ws, gpu_compute_target_2)
    print("Found existing compute target: {}".format(gpu_compute.name))
except:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6S_V3",
                                                                min_nodes = 0,
                                                                max_nodes = 4)
    gpu_compute_2 = AmlCompute.create(ws, gpu_compute_target_2, provisioning_config)
    gpu_compute_2.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    print("Created new compute target: {}".format(gpu_compute_2.name))

Found existing compute target: cpucluster
Found existing compute target: gpuclusterNCv2
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
Created new compute target: gpuclusterNCv3


In [45]:
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData

fs = Datastore.get(ws, datastore_name='workspacefilestore')
blob_storage = Datastore.get(ws, datastore_name='workspaceblobstore')

# Setting up the pipeline 

# Step 1 - Generate poem from finetuned GPT-2 Model

# Mapping the data reference of trained model 

In [46]:
trained_lang_model = DataReference(
    datastore=fs,
    data_reference_name="finetunedGPTModel",
    path_on_datastore="finetunedGPTModel/run1")

generatedHaiku = PipelineData(name="generatedHaiku", 
                            datastore=blob_storage)

In [47]:
from azureml.pipeline.steps import EstimatorStep
from azureml.train.estimator import Estimator

from azureml.pipeline.core.graph import PipelineParameter
from azureml.core.runconfig import CondaDependencies, RunConfiguration
from azureml.pipeline.steps import MpiStep

project_folder = "../Auria/gpt-2-finetuning/"

genHaikuEst = Estimator(source_directory=project_folder,
                         compute_target=gpu_compute_target_1,
                         entry_script='generateFromGPT.py',
                         node_count=1,
                         process_count_per_node=1,
                         pip_packages=[ "gpt-2-simple", "numpy", "tensorflow-gpu==1.13.1"],
                         custom_docker_image = "sleebapaul/cuda_10_0_cudnn_7_conda:ubuntu",
                         use_gpu=True)
genHaikuEst._estimator_config.environment.python.user_managed_dependencies=False


est_step = EstimatorStep(name="Estimator_GPT_generation", 
                         estimator=genHaikuEst, 
                         estimator_entry_script_arguments=["--run_name", trained_lang_model, 
                                                           "--output_folder", generatedHaiku],
                         runconfig_pipeline_params=None, 
                         inputs=[trained_lang_model], 
                         outputs=[generatedHaiku], 
                         compute_target=gpu_compute_target_1,
                         allow_reuse=False)

print("genHaikuStep step created")

genHaikuStep step created


# Custom docker image - sleebapaul/cuda_10_0_cudnn_7_conda:ubuntu

# pip packages used - gpt-2-simple, numpy, tensorflow-gpu==1.13.1

# Input - trained_lang_model

# Output - generatedHaiku 

# Project folder - gpt-2-finetuning

# Script name - generateFromGPT.py

# Step two 

# Generating the image from text 

# Mapping data references 
    

In [48]:
coco_AttnGAN2 = DataReference(
    datastore=blob_storage,
    data_reference_name="coco_AttnGAN2",
    path_on_datastore="sleebaBlobs/coco_AttnGAN2.pth")

image_encoder100 = DataReference(
    datastore=blob_storage,
    path_on_datastore="sleebaBlobs/image_encoder100.pth")

text_encoder100 = DataReference(
    datastore=blob_storage,
    data_reference_name="text_encoder100",
    path_on_datastore="sleebaBlobs/text_encoder100.pth")

data_dir = DataReference(
    datastore=fs,
    data_reference_name="coco",
    path_on_datastore="coco")


generatedArt = PipelineData(name="generatedArt", datastore=blob_storage)

In [49]:
project_folder = "../Auria/attnGAN/"

nodecount_param = PipelineParameter(name="nodecount", default_value=3)

cd = CondaDependencies()
cd.add_channel("conda-forge")
cd.add_channel("pytorch")
cd.add_conda_package("pytorch")
cd.add_conda_package("torchvision")

# Runconfig
amlcompute_run_config = RunConfiguration(conda_dependencies=cd)
amlcompute_run_config.environment.docker.enabled = True
amlcompute_run_config.environment.docker.gpu_support = True
amlcompute_run_config.environment.docker.base_image = "pytorch/pytorch"
amlcompute_run_config.environment.spark.precache_packages = False

textToImageStep = MpiStep(
    name="text_to_image_step",
    script_name="gen_art.py",
    arguments=["--gpu", 0,  
               "--data_dir", data_dir, 
               "--model_path", coco_AttnGAN2,
               "--textencoder_path", text_encoder100,
               "--input_text_folder", generatedHaiku,
               "--output_dir", generatedArt
              ],
    inputs=[data_dir, coco_AttnGAN2, text_encoder100, generatedHaiku],
    outputs=[generatedArt],
    compute_target=gpu_compute_1,
    node_count=nodecount_param, 
    process_count_per_node=1,
    pip_packages=["mpi4py", "torch", "torchvision", "numpy", "pyyaml", "nltk","easydict","pandas", "Pillow" ,"scikit-image"],
    source_directory=project_folder,
    runconfig=amlcompute_run_config,
    use_gpu=True,
    allow_reuse=False)

print("textToImageStep step created")


WARNING - 'distributed_backend' parameter will be deprecated. Please use 'distributed_training' instead.


textToImageStep step created


# Step three

# Photo styling 

In [50]:
photo_wct = DataReference(
    datastore=blob_storage,
    data_reference_name="photo_wct",
    path_on_datastore="sleebaBlobs/photo_wct.pth")

wikiart_emotions = DataReference(
    datastore=fs,
    data_reference_name="WikiArt_Emotions",
    path_on_datastore="WikiArt-Emotions")

coloredImage = PipelineData(name="coloredImage", 
                                    datastore=blob_storage)

print("DataReference object created")

DataReference object created


In [51]:
project_folder = "../Auria/NVIDIAFastPhotoStyle/"

coloringEst = Estimator(source_directory=project_folder,
                      compute_target=gpu_compute_target_1,
                      entry_script='demo.py',
                      node_count=1,
                      process_count_per_node=1,
                      pip_packages=[ "http://download.pytorch.org/whl/cu91/torch-0.4.0-cp36-cp36m-linux_x86_64.whl", 
                                    "torchvision==0.2.1", "scikit-umfpack", "setuptools", "numpy","Pillow", "scipy==1.1.0",
                                    "pynvrtc==9.1", "cupy-cuda91"],
                      custom_docker_image = "sleebapaul/cuda_9_1_conda:ubuntu",
                      use_gpu=True)
coloringEst._estimator_config.environment.python.user_managed_dependencies=False


coloringeEstStep = EstimatorStep(name="Estimator_coloring_step", 
                         estimator=coloringEst, 
                         estimator_entry_script_arguments=["--content_image_path", generatedArt, 
                                                           "--model_path", photo_wct,
                                                          "--style_image_path", wikiart_emotions,
                                                          "--output_image_path", coloredImage],
                         runconfig_pipeline_params=None, 
                         inputs=[photo_wct, wikiart_emotions, generatedArt], 
                         outputs=[coloredImage], 
                         compute_target=gpu_compute_target_1,
                         allow_reuse=False)

print("Coloring step created")

Coloring step created


# Step 4

# Setting up the pipeline with 3 steps 

In [52]:
from azureml.pipeline.core import Pipeline

steps = [est_step, textToImageStep, coloringeEstStep]

auria_pipeline = Pipeline(workspace=ws, steps=steps)
print("Pipeline is built")

auria_pipeline.validate()
print("Pipeline validation complete")

Pipeline is built
Step Estimator_GPT_generation is ready to be created [2276a757]
Step text_to_image_step is ready to be created [011807ad]
Step Estimator_coloring_step is ready to be created [d0777e9c]
Pipeline validation complete


In [53]:
from azureml.core import Experiment

pipeline_exp = Experiment(ws,'Auria_Pipeline_Three').submit(auria_pipeline)

Created step Estimator_GPT_generation [2276a757][219d944f-8755-424d-9bcd-4ca953d2bc2a], (This step will run and generate new outputs)
Created step text_to_image_step [011807ad][084a4455-76ca-424e-9ac9-da5dab4865d7], (This step will run and generate new outputs)
Created step Estimator_coloring_step [d0777e9c][4e2cb8d6-03fc-43b5-a5a3-26d25f5bac8f], (This step will run and generate new outputs)
Using data reference finetunedGPTModel for StepId [f567eedf][290212f9-9822-4507-b12d-f94c7b4b2969], (Consumers of this data are eligible to reuse prior runs.)
Using data reference coco for StepId [4950fee6][3b008c6a-fd9f-4adb-9065-5d40e43a2cfd], (Consumers of this data are eligible to reuse prior runs.)
Using data reference coco_AttnGAN2 for StepId [12901955][8a824f1e-c48b-4371-a335-2a754ca944bb], (Consumers of this data are eligible to reuse prior runs.)
Using data reference text_encoder100 for StepId [84ce7549][bb750316-709e-4125-8165-204cb1bea2af], (Consumers of this data are eligible to reuse p

# Step 5 

# Publishing it as a pipeline 

In [54]:
published_training_pipeline = auria_pipeline.publish(name="Auria_Pipeline_SP",
                                                        description="This new pipeline is the creative pursuit of Auria")
print("The published pipeline ID is {}".format(published_training_pipeline.id))

The published pipeline ID is 25ab721b-1c39-444f-9730-7a045611af65


In [55]:
from azureml.widgets import RunDetails
RunDetails(pipeline_exp).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …